# ETL

## Importar las librerías necesarias.

In [1]:
import pandas as pd 
import numpy as np
from datetime import datetime

## Cargar los datos

In [2]:
df = pd.read_csv('AccidentesAviones.csv')
df.sample(1)

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
1128,1128,"August 09, 1949",?,"Salalah, Oman",Military - Royal Air Force,?,?,Douglas C-47B,KN336,15901/32649,12,?,?,12,?,?,0,Crashed short of the runway.


### **Diccionario de datos:**
- ***Unnamed: 0:*** Número de identificación o índice para cada registro de accidente de vuelo
- ***fecha:*** Fecha en la que ocurrió el accidente de vuelo
- ***HORA declarada:*** Hora declarada del accidente aéreo
- ***Ruta:*** Ubicación geográfica donde ocurrió el accidente de vuelo
- ***OperadOR:*** Compañía aérea operadora del vuelo
- ***flight_no:*** Número de vuelo
- ***route:*** Ruta del vuelo, indicando los puntos de partida y destino
- ***ac_type:*** Tipo de aeronave involucrada en el accidente
- ***registration:*** Número de registro o matrícula de la aeronave
- ***cn_ln:*** Número de serie de la aeronave
- ***all_aboard:*** Total de personas a bordo (pasajeros y tripulación)
- ***PASAJEROS A BORDO:*** Total de pasajeros a bordo
- ***crew_aboard:*** Total de tripulación a bordo
- ***cantidad de fallecidos:*** Total de personas fallecidas en el accidente (pasajeros y tripulación)
- ***passenger_fatalities:*** Total de pasajeros fallecidos en el accidente
- ***crew_fatalities:*** Total de tripulantes fallecidos en el accidente
- ***ground:*** Total de personas en tierra fallecidas en el accidente
- ***summary:*** Resumen del accidente de vuelo y las circunstancias asociadas

## Transformaciones

### Cambiamos el tipo de dato de cada columna según su variable.

In [4]:
# Colummna: 'fecha'
df['fecha'] = pd.to_datetime(df['fecha'])  # Cambiar a tipo de datos fecha

In [6]:
# Colummna: 'HORA declarada'
def parse_hour(x):
    if x is None:
        return None
    if x.startswith('c '):
        x = x[2:]  # Eliminar el prefijo 'c '
    try:
        if len(x) == 4:
            return datetime.strptime(x, "%H%M").strftime("%H:%M")
        else:
            return datetime.strptime(x[:-1], "%H:%M").strftime("%H:%M")
    except ValueError:
        return None

def convert_to_time(x):
    if x is None:
        return None
    if x.startswith('c '):
        x = x[2:]  # Eliminar el prefijo 'c '
    try:
        return datetime.strptime(x[:-1], "%H:%M").strftime("%H:%M")
    except ValueError:
        return None

def remove_z_from_time(x):
    if x is None:
        return None
    if x.endswith('Z'):
        return x[:-1]
    else:
        return x

def remove_c_prefix(x):
    if x is None:
        return None
    if x.startswith('c '):
        return x[2:]
    else:
        return x

# Aplicar parse_hour y convert_to_time a los valores incorrectos
mask = df['HORA declarada'].apply(parse_hour).isna()
df.loc[mask, 'HORA declarada'] = df.loc[mask, 'HORA declarada'].apply(convert_to_time)

# Asegurar que los valores en 'HORA declarada' tengan el formato HH:MM
df['HORA declarada'] = df['HORA declarada'].apply(lambda x: x[:2] + ':' + x[2:] if x is not None and len(x) == 4 else x)

# Remover 'Z' anidada de los valores
df['HORA declarada'] = df['HORA declarada'].apply(remove_z_from_time)

# Eliminar el prefijo 'c ' de los valores
df['HORA declarada'] = df['HORA declarada'].apply(remove_c_prefix)

# convierte la columna "HORA declarada" en formato de texto a formato de fecha y hora
df['HORA declarada'] = pd.to_datetime(df['HORA declarada'], format='%H:%M', errors='coerce')

# Extraemos la hora y lo guarda en la columna "HORA declarada" como un número entero
df['HORA declarada'] = df['HORA declarada'].apply(lambda x: x.hour if pd.notnull(x) else None)

In [7]:
# Colummna: 'Ruta'
# Etiquetar los datos '?' en la columna de 'Ruta' como 'NaN'
#df.loc[df['Ruta'] == '?', 'Ruta'] = np.nan
df['Ruta'] = df['Ruta'].replace('?', pd.NA)

In [8]:
# Colummna: 'OperadOR'
# Etiquetar los datos '?' en la columna de 'OperadOR' como 'NaN'
#df.loc[df['OperadOR'] == '?', 'OperadOR'] = np.nan
df['OperadOR'] = df['OperadOR'].replace('?', pd.NA)

In [9]:
# Colummna: 'flight_no'
df['flight_no'] = df['flight_no'].replace('?', pd.NA) 

In [10]:
# Colummna: 'route'
df['route'] = df['route'].replace('?', pd.NA)
df['route'] = df['route'].where(df['route'].str.contains('-'), other=pd.NA)

In [11]:
# Colummna: 'ac_type'
df['ac_type'] = df['ac_type'].replace('?', pd.NA)
df['ac_type'] = df['ac_type'].replace('Unknown / Unknown', pd.NA)

In [12]:
# Colummna: 'registration'
df['registration'] = df['registration'].replace('?', pd.NA)

In [13]:
# Colummna: 'cn_ln'
df['cn_ln'] = df['cn_ln'].replace('?', pd.NA)

In [14]:
# Colummna: 'all_aboard'
df['all_aboard'] = pd.to_numeric(df['all_aboard'], errors='coerce')

In [15]:
# Colummna: 'PASAJEROS A BORDO'
df['PASAJEROS A BORDO'] = pd.to_numeric(df['PASAJEROS A BORDO'], errors='coerce')  # Cambiar a tipo de dato entero y manejar los valores no numéricos

In [16]:
# Colummna: 'crew_aboard'
df['crew_aboard'] = pd.to_numeric(df['crew_aboard'], errors='coerce')  # Cambiar a tipo de dato entero y manejar los valores no numéricos

In [17]:
# Colummna: 'cantidad de fallecidos'
df['cantidad de fallecidos'] = pd.to_numeric(df['cantidad de fallecidos'], errors='coerce')  # Cambiar a tipo de dato entero y manejar los valores no numéricos

In [18]:
# Colummna: 'passenger_fatalities'
df['passenger_fatalities'] = pd.to_numeric(df['cantidad de fallecidos'], errors='coerce')  # Cambiar a tipo de dato entero y manejar los valores no numéricos

In [19]:
# Colummna: 'crew_fatalities'
df['crew_fatalities'] = pd.to_numeric(df['crew_fatalities'], errors='coerce')  # Cambiar a tipo de dato entero y manejar los valores no numéricos

In [20]:
# Colummna: 'ground'
df['ground'] = pd.to_numeric(df['ground'], errors='coerce')  # Cambiar a tipo de dato entero y manejar los valores no numéricos

In [21]:
# Colummna: 'summary'
df['summary'] = df['summary'].replace('?', pd.NA)

In [24]:
# Exportamos a un dataset final de nombre 'accidentes_etl' para el EDA
df.to_csv('accidentes_etl.csv', index=False)